In [11]:

import os 
import numpy as np
import time

In [8]:
def spt(instance, job_num, machine_num, bias=None):
    """
    for non-flexible job shop

    :param instance: [time, order]
    :param job_num:
    :param machine_num:
    :param bias: dynamic use
    :return:
    """
    time = instance[0]
    order = instance[1]
    start_time = np.zeros((job_num, machine_num), dtype=np.int32)
    end_time = np.zeros((job_num, machine_num), dtype=np.int32)

    machine_process = [[] for _ in range(machine_num)]  # 记录每个机器上任务的执行情况 (job id, task id in job)
    candidates = []  # 记录可被调度的任务信息 (job_id, task_id_in_job, machine id, processing time)

    for i in range(job_num):
        candidates.append((i, 0, order[i, 0] - 1, time[i, 0]))

    # 初始化
    # for i in range(job_num):
    #     candidates.append((i, 0, order[i, 0]-1, time[i, 0]))
    task_seq = []
    # for k in range(machine_num):
    #     candidates = []
    #     for i in range(job_num):
    #         candidates.append((i, k, order[i, k] - 1, time[i, k]))
    #
    #     candidates_sorted = sorted(candidates, key=lambda x: x[-1])
    while True:
        candidates_s = sorted(candidates, key=lambda x: x[-1])
        p = candidates_s[0]
        # for p in candidates_sorted:
        if p[-1] == np.inf:
            break

        job_id = p[0]
        task_id_in_job = p[1]
        machine_id = p[2]
        processing_time = p[3]

        action = job_id*machine_num + task_id_in_job

        # [[action], [machine_id], [job_id], [task_id], [duration]]
        # task_seq.append(action)
        task_seq.append([[action], [machine_id], [job_id], [task_id_in_job], [processing_time]])

        # 主要就是确定任务开始时间
        # if len(machine_process[machine_id]) == 0:
        #     if task_id_in_job == 0:
        #         start_time[job_id, task_id_in_job] = 0
        #     else:
        #         start_time[job_id, task_id_in_job] = end_time[job_id, task_id_in_job-1]   # 同一任务前置任务结束时可开始
        # else:
        #     pre_task_in_machine = machine_process[machine_id][-1]  # job_id 0, task_id 1
        #     if task_id_in_job == 0:
        #         start_time[job_id, task_id_in_job] = end_time[pre_task_in_machine[0], pre_task_in_machine[1]]
        #     else:
        #         start_time[job_id, task_id_in_job] = max(end_time[job_id, task_id_in_job-1],
        #                                                  end_time[pre_task_in_machine[0], pre_task_in_machine[1]])
        #
        # end_time[job_id, task_id_in_job] = start_time[job_id, task_id_in_job] + processing_time
        # machine_process[machine_id].append(p)

        # 更新candidate
        if task_id_in_job < machine_num-1:
            next_in_job = task_id_in_job+1
            candidates[job_id] = (job_id, next_in_job, order[job_id, next_in_job]-1, time[job_id, next_in_job])
        else:
            candidates[job_id] = (job_id, task_id_in_job+1, np.inf, np.inf)

    return task_seq


def mwr(instance, job_num, machine_num):
    """
    for non-flexible job shop
    最长剩余工作时间优先.

    :param instance: [time, order]
    :param job_num:
    :param machine_num:
    :return:
    """

    time = instance[0]
    order = instance[1]

    candidates = []  # 记录可被调度的任务信息 (job_id, task_id_in_job, machine id, processing time，remain work time this job)

    for i in range(job_num):
        candidates.append((i, 0, order[i, 0] - 1, time[i, 0], np.sum(time[i])))

    task_seq = []

    while True:
        candidates_s = sorted(candidates, key=lambda x: x[-1])
        p = candidates_s[-1]
        # for p in candidates_sorted:
        if p[-1] == -np.inf:
            break

        job_id = p[0]
        task_id_in_job = p[1]
        machine_id = p[2]
        processing_time = p[3]
        remain_time = p[4]

        action = job_id * machine_num + task_id_in_job
        # task_seq.append(action)
        task_seq.append([[action], [machine_id], [job_id], [task_id_in_job], [processing_time]])

        if task_id_in_job < machine_num-1:
            next_in_job = task_id_in_job+1
            candidates[job_id] = (job_id, next_in_job, order[job_id, next_in_job]-1, time[job_id, next_in_job],
                                  remain_time)
        else:
            candidates[job_id] = (job_id, task_id_in_job+1, -np.inf, -np.inf, -np.inf)

    return task_seq


def mor(instance, job_num, machine_num):
    """
    for non-flexible job shop
    最多剩余工序优先

    :param instance: [time, order]
    :param job_num:
    :param machine_num:
    :return:
    """

    time = instance[0]
    order = instance[1]

    candidates = []

    for i in range(job_num):
        candidates.append((i, 0, order[i, 0] - 1, time[i, 0], machine_num))

    task_seq = []

    while True:
        c = candidates.copy()
        np.random.shuffle(c)
        candidates_s = sorted(c, key=lambda x: x[-1])
        p = candidates_s[-1]
        # for p in candidates_sorted:
        if p[-1] == -np.inf:
            break

        job_id = p[0]
        task_id_in_job = p[1]
        machine_id = p[2]
        processing_time = p[3]
        remain_task_num = p[4]

        action = job_id * machine_num + task_id_in_job
        # task_seq.append(action)
        task_seq.append([[action], [machine_id], [job_id], [task_id_in_job], [processing_time]])

        if task_id_in_job < machine_num-1:
            next_in_job = task_id_in_job+1
            candidates[job_id] = (job_id, next_in_job, order[job_id, next_in_job]-1, time[job_id, next_in_job],
                                  remain_task_num)
        else:
            candidates[job_id] = (job_id, task_id_in_job+1, -np.inf, -np.inf, -np.inf)

    return task_seq


In [13]:
def task_seq_to_makespan(jobs, task_seq, job_num, machine_num):
    machine_record = [[] for _ in range(machine_num)]
    end_time = np.zeros((job_num, machine_num))

    for task in task_seq:
        task_id = task[0][0]
        machine_id = task[1][0]
        job_id = task[2][0]
        task_order_in_job = task[3][0]
        duration = task[4][0]

        # task_order_in_job = task_id % machine_num
        # 计算该task的开始时间
        # 先从前置任务计算开始时间的下界
        if task_order_in_job == 0:
            start_time = 0
        else:
            start_time = end_time[job_id, task_order_in_job-1]

        machine_status = machine_record[machine_id]
        idx = 0
        for record in machine_status:
            if start_time >= record[0]:
                # 该任务开始时间晚于当前任务的开始时间
                start_time = max(start_time, record[1])     # 则该任务开始时间的下界不早于当前任务的结束时间
                idx += 1
                continue
            elif start_time+duration <= record[0]:
                # 此处可插入
                break
            else:
                # start time早于当前任务开始时间，但是空隙不够插入，则开始时间下界为当前任务结束时间
                start_time = record[1]
                idx += 1
                continue

        machine_status.insert(idx, (start_time, start_time+duration))
        end_time[job_id, task_order_in_job] = start_time+duration

    makespan = np.max(end_time[:, -1])
    return makespan, machine_record

In [17]:

def heuristic_test():
    # instances = np.load("../Instances/Benchmark/ta/J30_M20.npy")
    # from Env.SJSSPEnv import SJsspEnv
    # env = SJsspEnv(15, 15)

    # task_seq = spt(instances, 15, 15)
    # makespan, end_times = task_seq_to_makespan(None, task_seq, 15, 15)

    # method = spt
    # save_path = f"../TestLog/gen/{method.__name__}"  # 1
    is_dynamic = False
    print(os.getcwd())
    # (job_num, machine_num, high_bound)
    # params = [(6, 6, 100), (10, 10, 100), (15, 10, 100), (40, 30, 100), (50, 30, 100), (100, 30, 100), (20, 15, 50),
    #           (20, 15, 150), (20, 15, 300)]

    set_names = ['ta', 'dmu']
    set_sizes = {'ta': [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)],
                 'dmu': [(20, 15), (20, 20), (30, 15), (30, 20), (40, 15), (40, 20), (50, 15), (50, 20)]}

    set_dmu = [(20, 15), (20, 20), (30, 15), (30, 20), (40, 15), (40, 20), (50, 15), (50, 20)]

    for method in (spt, mwr, mor):
        print(method.__name__)
        save_path = f"."
        # record = {}
        # for size in [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]:   # 4
        for param in set_dmu:
            job_num = param[0]
            machine_num = param[1]
            # high_bound = param[2]

            instances = np.load(f"./J{job_num}_M{machine_num}.npy")  # 5
            ins = np.load('J20_M15.npy')

            num = instances.shape[0]
            objs = []
            times = []
            orders = []
            for i in range(num):
                instance = instances[i]

                start = time.time()
                order = method(instance, job_num, machine_num)  # 2
                obj, _ = task_seq_to_makespan(None, order, job_num, machine_num)
                end = time.time()

                times.append(end - start)
                orders.append(order)

                objs.append(obj)

            # print(f"gen J{job_num} M{machine_num} done.")
            # print(np.mean(times))
            # np.save(f"./J{job_num}_M{machine_num}_res.npy", times)
            print(np.mean(objs))
            print()

    # for sigma in [10, 20]:
    #     print(sigma)
    #     for method in (spt, mwr, mor):
    #         print(method.__name__)
    #         for name in set_names:
    #             save_path = f"../TestLog/{name}/dynamic/{sigma}/{method.__name__}"
    #             # record = {}
    #             # for size in [(15, 15), (20, 15), (20, 20), (30, 15), (30, 20), (50, 15), (50, 20), (100, 20)]:   # 4
    #             for param in set_sizes[name]:
    #                 job_num = param[0]
    #                 machine_num = param[1]
    #                 # high_bound = param[2]
    
    #                 # name = f"gen_J{job_num}_M{machine_num}_H{high_bound}"
    #                 instances = np.load(f"../Instances/Benchmark/{name}/J{job_num}_M{machine_num}.npy")  # 5
    #                 # instances = np.load("../Instances/Benchmark/ta/J30_M20.npy")
    #                 # job_num = 30
    #                 # machine_num = 20
    
    #                 num = instances.shape[0]
    #                 if is_dynamic:
    #                     biases = np.load(f"../Instances/Benchmark/{name}/bias/{sigma}/J{job_num}_M{machine_num}.npy")     # 6
    
    #                 objs = []
    #                 times = []
    #                 orders = []
    #                 for i in range(num):
    #                     instance = instances[i]
    
    #                     if is_dynamic:
    #                         bias = biases[i]
    
    #                     start = time.time()
    #                     order = method(instance, job_num, machine_num)  # 2
    #                     obj, _ = task_seq_to_makespan(None, order, job_num, machine_num)
    #                     end = time.time()
    
    #                     times.append(end - start)
    #                     orders.append(order)
    
    #                     # env = SJsspEnv(job_num, machine_num)
    #                     # env.reset(instance)
    #                     # for o in order:
    #                     #     env.step(o)
    #                     #     # end_time.append(env.max_end_time)
    #                     #
    #                     # obj = env.max_end_time
    #                     if is_dynamic:
    #                         obj += np.sum(biases[i])
    #                     objs.append(obj)
    
    #                 # record[name] = np.mean(objs)
    
    #                 print(name)
    #                 # print(objs)
    #                 print(np.mean(times))
    #                 # np.save(f"{save_path}/{name}_solu.npy", objs)
    #                 np.save(f"{save_path}/J{job_num}_M{machine_num}_time.npy", times)
    #                 # np.save(f"../Instances/DataGen/solu/{method.__name__}/{name}.npy", orders)  # 3
    #                 #
    #                 print(name + f"J{job_num} M{machine_num} sigma {sigma} done.")
    #                 print()
    
    #                 # obj = heuristicSPT(30, 15, instance[1]-1, instance[0])
    
    #             # np.save(f"gen_{method.__name__}.npy", record)
    #

In [18]:
heuristic_test()

/home/nbicc/data/hhb/DFJSSPLib/DFJSSPLib 2/Instances/Benchmark/dmu
spt
4801.9

5691.0

6301.1

6876.4

7604.4

8478.1

8933.4

9915.1

mwr
4569.7

5289.4

6070.2

6727.8

7164.2

7982.6

8449.0

9506.3

mor
4523.0

5012.8

5742.9

6561.7

7258.6

7950.3

8650.2

9286.7

